*In this tutorial we will investigate addresses with a large number of companies registered via the API and Companies House Data Product download.*

### Busy Addresses and API Limits

In [ ]:
from sugartrail import base, api, mapview
import pandas as pd
api.basic_auth.username = ""

When navigating Companies House there are times that we will run into some very popular addresses. For example lets say build a network from [this officer](https://find-and-update.company-information.service.gov.uk/officers/Nd2URspq4bvLy-hwzDZ0_p7FGJw/appointments):

In [ ]:
officer_id = "Nd2URspq4bvLy-hwzDZ0_p7FGJw"
network = base.Network(officer_id=officer_id)
network.perform_hop(2)

Within 2 hops we've got over 60 addresses (although many of them look like duplicate entries):

In [ ]:
pd.DataFrame(network.addresses)

In [ ]:
pd.DataFrame(network.addresses)['address'].unique()

If we check out the `maxsize_entities` property of our Network class, we will see a dataframe containing all of the addresses and officers that have exceeded the maxsize limits imposed in the Hop class. In this case, we can see one of the addresses in the network has over 4800 companies based there.

In [ ]:
pd.DataFrame(network.maxsize_entities)

Because we set a limit of 50 companies on the maxsize of companies returned via `companies_at_address_maxsize`, these companies will not be added to `companies_id`.

In [ ]:
network.hop.companies_at_address_maxsize

If we check `companies_id` we'll notice it hasn't had 4800 companies added to it:

In [ ]:
len(network.company_ids)

Including limits is useful to avoid our databases getting clogged up with random companies. 
Although lets pause to briefly explore what address would have thousands of companies registered there?

In [ ]:
network.maxsize_entities[2]['node']

![title](../assets/images/regent_storefront.jpeg)

"3rd Floor, 207, Regent Street" is a "virtual office" run by a company called [Hold Everything](https://www.hold-everything.com/). Businesses can use this address for correspondance/registration for £24 a month:

![title](../assets/images/exclusive.png)

However the large number of companies registered at a single address can lead to many instances of mistaken identity. Just because a company is registered at a virtual office does not mean it has any connection with other companies registered there.:

![title](../assets/images/review.png)

Numerous media outlets have reported on fraudulent companies that use virtual offices and incorporation services: 
- Kemp House, 162 City Road | Capital Officer: [Mystery group took millions in furlough funds - Financial Times](https://www.ft.com/content/b3c70369-5170-47ca-b779-fc0898fd29e6)
- 20-22 Wenlock Road | Made Simple: [Court shuts down companies behind £9m truffle scam - Gov.uk](https://www.gov.uk/government/news/court-shuts-down-companies-behind-9m-truffle-scam)
- 2 Woodberry Down | A1 Company Services [How A Suburban North London House Is Connected To The Paul Manafort Indictment - Huffington Post](https://www.huffingtonpost.co.uk/entry/manfort-london-connection_uk_59f72f50e4b07fdc5fbf92c7)
- 29 Harley Street | Formations House [Offshore in central London: the curious case of 29 Harley Street - The Guardian](https://www.theguardian.com/business/2016/apr/19/offshore-central-london-curious-case-29-harley-street)
- 63-66 Hatton Garden | Valemont Properties Ltd [The Global Laundromat: how did it work and who benefited? - The Guardian](https://www.theguardian.com/world/2017/mar/20/the-global-laundromat-how-did-it-work-and-who-benefited)

If we wanted to get all companies listed at 207 Regent Street we can adjust our maxsize limits to `None` and attempt to perform a hop again:

In [ ]:
regent_street_network = base.Network(address='3rd Floor, 207 Regent Street London W1B 3HH England')
regent_street_network.hop.companies_at_address_maxsize = None
regent_street_network.hop.officers_at_address_maxsize = None
regent_street_network.perform_hop(1)

In [ ]:
pd.DataFrame(regent_street_network.company_ids)

Such large networks can still be interesting to analyse. For instance if we perform another hop this will get all the officers for every company at the address. This will take several hours to build as we have lots of companies to analyse, however if we want to save time we could just uncomment and load a pre-made network below:  

In [ ]:
regent_street_network.perform_hop(1)

In [ ]:
regent_street_network = base.Network(file='regent_street_network.json')

Analysing the most frequently occuring officers running businesses from 207 Regent Street returns some very busy officers and incorporation agents:

In [ ]:
pd.DataFrame(regent_street_network.officer_ids)['name'].value_counts()

A quick news lookup on two of the officers in the top 5, J. Beardsley of Helve TCS Limited and S. Poppleton reveal these names to be connected to several known instances of fraud:
- [Fraudster duo jailed for their part in defrauding millions of pounds from over 100 victims - Crown Prosecution Service](https://www.cps.gov.uk/cps/news/fraudster-duo-jailed-their-part-defrauding-millions-pounds-over-100-victims)
- [Print farming companies struck off - Printweek](https://www.printweek.com/news/article/print-farming-companies-struck-off)
- [Rogue book publishers slammed shut by the courts - Gov.uk](https://www.gov.uk/government/news/rogue-book-publishers-slammed-shut-by-the-courts)

### Busier Addresses and Downloaded Data

There are situations where some addresses have thousands or even tens of thousands of companies registered. Companies House provides two methods for getting company data, API and data product. We used the API to get the information above which returns all active and dissolved companies registered to the address. We get the same result when we attempt to perform an advanced company search using this address through the website:

![title](../assets/images/regent.png)

Unfortunately the API is limited to returing 5000 result max. This is fine in our case with 207 Regent Street because we're just under the limit. However there are much bigger fish out there for instance, '75 Shelton Street':

In [ ]:
shelton_street_network = base.Network(address="71-75, Shelton Street, Covent Garden, London, WC2H 9JQ")
shelton_street_network.perform_hop(1)
shelton_street_network.maxsize_entities[0]

We can already see its over 5000 limit for the API. If we check online we can see the number is huge: 

![title](../assets/images/shelton.png)

This is where the data product comes in. We can download it in one go and use it to get all of the "active" companies. To use the data product:
1. Download it from [here](http://download.companieshouse.gov.uk/en_output.html) (might take some time as its a pretty large file ~430Mb)
2. Move it to local directory `assets/company_data/` and unzip the file 
3. Load into a dataframe which we can pass to our network class

Might take a minute to load. How adjust the file string below and attempt to load it into `company_data`:

In [ ]:
company_data = pd.read_csv("../assets/company_data/BasicCompanyDataAsOneFile-2022-11-01.csv")

Now lets try get every company at the very overcrowded 71-75 Shelton Street address (might take several minutes- can uncomment the cell below to load pre-made network):

In [ ]:
shelton_street_network = base.Network(address="71-75, Shelton Street, Covent Garden, London, WC2H 9JQ")
shelton_street_network.hop.companies_at_address_maxsize = None
shelton_street_network.hop.officers_at_address_maxsize = None
shelton_street_network.get_officers_at_address = False
shelton_street_network.perform_hop(1, company_data = company_data)

In [ ]:
shelton_street_network = base.Network(file='shelton_street_network.json')

If we check `company_ids` we have over 70000 companies that we could build a network from if we had lots of time on our hands:

In [ ]:
len(shelton_street_network.company_ids)